# Fundamentos da Neuropedagogia Computacional
    This file is part of program Activ Spyder
    Copyright © 2022 Carlo Oliveira carlo@nce.ufrj.br,
    Labase <http://labase.selfip.org/>; GPL <http://is.gd/3Udt>.
    SPDX-License-Identifier: (GPLv3-or-later AND LGPL-2.0-only) WITH bison-exception

## Análise dos dados do documento de especificação -
    Inpeciona e gera estatísticas sobre o texto de especificação.
[!NOTE]
codeauthor:: Carlo Oliveira [carlo at ufrj](mailto:carlo@ufrj.br)

## Changelog

<!--![version added](https://raw.githubusercontent.com/FortAwesome/Font-Awesome/6.x/svgs/solid/circle-plus.svg)  22.06-->
<img src="https://raw.githubusercontent.com/FortAwesome/Font-Awesome/6.x/svgs/solid/circle-plus.svg" width=16/>  22.08

* Criação do raspador de página.
* Desenho do grafo com os nós coloridos

<img src="https://raw.githubusercontent.com/FortAwesome/Font-Awesome/6.x/svgs/solid/circle-plus.svg" width=16/>  22.09

* Preparação para vértices coloridos.

## Obtendo os dados do Github
O arquivo de especificação é guardado na página [Conteúdo Processo](https://activufrj.nce.ufrj.br/wiki/NEUROPEDAGOGIA_COMPUTACIONAL_CONTEUDO_PROCESSO/Relatorio_1). Este arquivo foi Produzido pelos pesquisadores para especificar a área.

In [ ]:
STOPWORDS = "https://gitlab.com/cetoli/spike/-/raw/master/src/neurocomp/conf/stopwords.txt"
ESPECIFICA = "http://activufrj.nce.ufrj.br/raw/wiki/NEUROPEDAGOGIA_COMPUTACIONAL_CONTEUDO_PROCESSO/Relatorio_01"
"""Remove as stopwords da especificação"""
from urllib.request import urlopen
from collections import Counter
class Text:
    TEXT = None
    def __int__(self):
        Text.TEXT = self if not Text.TEXT else Text.TEXT
        self.original = None
        self.clean = None


with urlopen(ESPECIFICA) as response, urlopen(STOPWORDS) as stopword :
    tx = Text()
    Text.TEXT = tx
    body = response.read().lower().decode('utf-8')
    stop = stopword.read().decode('utf-8').split()
    stop.append("através")
    list_body = body.split()
    # print(list_body[:40])
    # clean = [u"{}".format(word) for word in list_body if word not in stop]
    clean = [word for word in list_body if (word not in stop)
             and ("\xa0" not in word) and ("<" not in word) and (word.isalpha())]
    # print(clean[:40])
    tx.clean = count = Counter(clean)
    tup = list(count.items())
    tup.sort(key = lambda x: x[1], reverse=True)
    print(tup[:100])

# Varredura do texto

O texto será percorrido usando uma janela de pertinência centrada em cada palavra

In [ ]:
class SurveyText:
    def __init__(self, text_dictionary, relevancy=2):
        def strip(w):
            return w[:-1] if w.endswith("s") else w
        self.text = text_dictionary
        self.relevant = [strip(word) for word, cnt in self.text.items() if cnt >= relevancy]
        self.pairs, self.levels = {}, {}

    def survey(self, window=10):
        size = len(self.relevant)
        for here, word in enumerate(self.relevant):
            start = here - window if here >= window else 0
            ender = here + window if here+window <= size else size
            for other in self.relevant[start:ender]:
                if other == word:
                    continue
                entry = (other, word) if other < word else (word, other)
                self.pairs[entry] = self.pairs.setdefault(entry, 0) + 1
        self.levels = Counter([i for (i, j), _ in self.pairs.items()])
        levels = Counter([j for (i, j), _  in self.pairs.items()])
        levels.update({key: value for key, value in self.levels.items() if key not in levels})
        levels = {key: (levels[key] + self.levels.setdefault(key, 0)) for key in levels}
        self.levels = {key: value//6-1 for key, value in levels.items() if value>8}

    def nodes(self, relevancy=2):
        # node = {name[0]: good for name, good in self.pairs.items() if good>relevancy}
        relevant = {na: good for (na, _), good in self.pairs.items() if good>relevancy}
        relevant.update({na: good for (_, na), good in self.pairs.items() if good>relevancy})
        node = {name: good for name, good in self.levels.items() if name in relevant}
        node = ({nd: cnt for nd, cnt in self.levels.items() if nd in node})
        # node.update({name[1]: good if good > node.setdefault(name[1], 0) else node[name[1]]
        #              for name, good in self.pairs.items() if good>relevancy})
        # return node.keys()
        # return node.items()
        # return self.levels.items()
        return node.items()

    def show(self, relevancy=2):
        tuple_list = list(self.pairs.items())
        # tuple_list = tuple_list.sort(key = lambda x: x[1], reverse=True)
        tuple_list = sorted(tuple_list, key = lambda x: x[1], reverse=True)
        _list = [(_tuple[0], _tuple[1], good) for _tuple, good in tuple_list if good>relevancy]
        return _list


st = SurveyText(Text.TEXT.clean)
st.survey(20)
print(st.show())  # [:200])
# print(st.nodes())

In [ ]:
import networkx as nx  # importing networkx package
import matplotlib.pyplot as plt # importing matplotlib package and pyplot is for displaying the graph on canvas
colors = {wg: cl for wg, cl in enumerate("yccmmbgrrrrrrrr")}
plt.rcParams["figure.figsize"] = [15.50, 8.50]
plt.rcParams["figure.autolayout"] = True
wg=nx.Graph()
_ = [wg.add_node(node, size=size) for node, size in st.nodes()]
_ = [wg.add_edge(a, b,color=colors[w], weight=w ) for a, b, w in st.show()]
sizes = [wg.nodes[s]['size']*100 for s in wg.nodes]
color = [colors[wg.nodes[s]['size']] for s in wg.nodes]

'''Node can be called by any python-hashable obj like string,number etc'''
nx.draw(wg, with_labels = True, node_size=sizes, node_color=color)  #draws the networkx graph containing nodes which are declared till before
plt.show()  # displays the networkx graph on matplotlib canvas


## Grafo em 3D
